![hopsworks_logo](../../images/hopsworks_logo.png)

# Part 02: Training Data & Feature views

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/fraud_batch/2_feature_view_creation.ipynb)

**Note**: you may get an error when installing hopsworks on Colab, and it is safe to ignore it.

This is the second part of the quick start series of tutorials about Hopsworks Feature Store. This notebook explains how to read from a feature group and create training dataset within the feature store

## 🗒️ In this notebook we will see how to create a training dataset from the feature groups:
1. **Select the features** we want to train our model on,
2. **How the features should be preprocessed,**
3. **Create a dataset split** for training and validation data.

![02_training-dataset](../../images/02_training-dataset.png)

#### <span style="color:#ff5f27;">📝 Importing Libraries</span>

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/164


Connected. Call `.close()` to terminate connection gracefully.


### <span style="color:#ff5f27;">⬇️ Data retrieving from Feature Groups</span>

We start by selecting all the features we want to include for model training/inference.

In [2]:
# Load feature groups.
fares_fg = fs.get_or_create_feature_group("fares_fg",
                                          version=1)
rides_fg = fs.get_or_create_feature_group("rides_fg",
                                          version=1)

In [3]:
# Select features for training data.
fg_query = fares_fg.select(['total_fare'])\
                            .join(rides_fg.select_except(['taxi_id', "driver_id", "pickup_datetime",
                                                          "pickup_longitude", "pickup_latitude",
                                                          "dropoff_longitude", "dropoff_latitude"]),
                                  on=['ride_id'])

fg_query.show(2)

2022-08-18 23:19:06,623 INFO: USE `romankah_featurestore`
2022-08-18 23:19:07,626 INFO: SELECT `fg1`.`total_fare` `total_fare`, `fg0`.`month_of_the_ride` `month_of_the_ride`, `fg0`.`ride_id` `ride_id`, `fg0`.`passenger_count` `passenger_count`, `fg0`.`distance` `distance`, `fg0`.`pickup_distance_to_jfk` `pickup_distance_to_jfk`, `fg0`.`dropoff_distance_to_jfk` `dropoff_distance_to_jfk`, `fg0`.`pickup_distance_to_ewr` `pickup_distance_to_ewr`, `fg0`.`dropoff_distance_to_ewr` `dropoff_distance_to_ewr`, `fg0`.`pickup_distance_to_lgr` `pickup_distance_to_lgr`, `fg0`.`dropoff_distance_to_lgr` `dropoff_distance_to_lgr`, `fg0`.`year` `year`, `fg0`.`weekday` `weekday`, `fg0`.`hour` `hour`
FROM `romankah_featurestore`.`fares_fg_1` `fg1`
INNER JOIN `romankah_featurestore`.`rides_fg_1` `fg0` ON `fg1`.`ride_id` = `fg0`.`ride_id`


,total_fare,month_of_the_ride,ride_id,passenger_count,distance,pickup_distance_to_jfk,dropoff_distance_to_jfk,pickup_distance_to_ewr,dropoff_distance_to_ewr,pickup_distance_to_lgr,dropoff_distance_to_lgr,year,weekday,hour
0,159.0,197001,da7f3c46970e2134e62443d1ba59adcf,2,57.191066,71.712117,51.754719,79.278673,41.917076,65.514943,41.128134,1970,0,13
1,70.0,197001,83558c7d52688b832ae37840945e86b1,4,66.124064,47.370011,83.881044,66.718941,90.097942,50.264600,77.147833,1970,0,12


#### <span style="color:#ff5f27;">〰️ Transformation Functions</span>


We can preprocess our data using several encoding methods like *min-max scaling* on numerical features and *label encoding* on categorical features. To do this we simply define a mapping between our features and transformation functions. This ensures that transformation functions such as *min-max scaling* are fitted only on the training data (and not the validation/test data), which ensures that there is no data leakage.

In [4]:
[t_func.name for t_func in fs.get_transformation_functions()]

['label_encoder', 'min_max_scaler', 'robust_scaler', 'standard_scaler']

In [5]:
# # Load transformation functions.
# min_max_scaler = fs.get_transformation_function(name="min_max_scaler")
# label_encoder = fs.get_transformation_function(name="label_encoder")

# # Map features to transformations.
# transformation_functions = {
#     "total_fare": min_max_scaler,
#     "distance": min_max_scaler
# }

## <span style="color:#ff5f27;">🔮 Feature View Creation</span>


We start by selecting all the features we want to include for model training/inference.

After we have made a query from desired features, we should make a corresponding `Feature View`.
In order to do it we may use `fs.create_feature_view()`

In [6]:
nyc_fares_fv = fs.create_feature_view(
    name='nyc_taxi_fares_fv',
    query=fg_query,
    labels=["total_fare"]
#     transformation_functions=transformation_functions
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/164/fs/106/fv/nyc_taxi_fares_fv/version/1


In [7]:
nyc_fares_fv.version

1

## <span style="color:#ff5f27;">🏋️ Training Dataset Creation</span>
    
In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

Training Dataset may contain splits such as:

    Training set - the subset of training data used to train a model.
    Validation set - the subset of training data used to evaluate hparams when training a model
    Test set - the holdout subset of training data used to evaluate a mode

Training dataset is created using fs.create_train_validation_test_split() method.

In [8]:
nyc_fares_fv.create_training_data(
    description='training_dataset',
    data_format='csv'
)

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/164/jobs/named/nyc_taxi_fares_fv_1_1_create_fv_td_18082022211938/executions


(1, <hsfs.core.job.Job at 0x27e687a1640>)

In [9]:
X_train, y_train = nyc_fares_fv.get_training_data(
    training_dataset_version=1
)

In [10]:
X_train.head(5)

,month_of_the_ride,ride_id,passenger_count,distance,pickup_distance_to_jfk,dropoff_distance_to_jfk,pickup_distance_to_ewr,dropoff_distance_to_ewr,pickup_distance_to_lgr,dropoff_distance_to_lgr,year,weekday,hour
0,197001,d070e81b13f9d4ee015365f50c2b2218,3,25.752523,22.054872,28.265240,30.532599,47.690248,15.095869,31.496596,1970,0,13
1,197001,2388bcea972106c742d21be4b0f18444,1,13.494440,27.150127,17.539545,45.999205,33.448316,29.572327,16.644308,1970,0,13
2,197001,8bf03ea97e74dfe125f10ba81f006b77,1,50.435481,29.150692,55.769077,49.973292,64.500894,35.215256,49.998029,1970,0,13
3,197001,422e728af278ddcf60039f87c523ecb0,4,56.774919,21.015145,70.865810,21.238427,77.968131,10.595304,64.438230,1970,0,13
4,197001,bdc3a865f7c4ec60f14faae016810848,1,22.729159,42.078924,54.268783,38.365793,57.656597,31.791766,46.069380,1970,0,14


In [11]:
y_train.head(5)

,total_fare
0,20.0
1,169.0
2,240.0
3,153.0
4,137.0


In [12]:
nyc_fares_fv.create_train_test_split(
    test_size=0.2 # here we can define the test dataset size
)

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/164/jobs/named/nyc_taxi_fares_fv_1_2_create_fv_td_18082022212036/executions


(2, <hsfs.core.job.Job at 0x27e687b5610>)

In [13]:
X_train, y_train, X_test, y_test = nyc_fares_fv.get_train_test_split(
    training_dataset_version=2
)

In [14]:
X_test.head(5)

,month_of_the_ride,ride_id,passenger_count,distance,pickup_distance_to_jfk,dropoff_distance_to_jfk,pickup_distance_to_ewr,dropoff_distance_to_ewr,pickup_distance_to_lgr,dropoff_distance_to_lgr,year,weekday,hour
0,197001,d070e81b13f9d4ee015365f50c2b2218,3,25.752523,22.054872,28.265240,30.532599,47.690248,15.095869,31.496596,1970,0,13
1,197001,1f609c564f968ca69591b3e7714c9692,3,63.851680,61.279378,40.410433,74.773954,30.644042,58.522090,29.825254,1970,0,14
2,197001,07645c3c6b9154d6e56ee7d293f9e00f,4,23.301742,74.100019,51.679051,87.120651,63.827367,71.076367,47.910363,1970,0,14
3,197001,da8f1aeb90ae756d4a37816b16754204,1,73.733474,83.076283,9.932646,72.789475,14.184656,72.450431,3.128552,1970,0,14
4,197001,761bb0add35b8a4d4c7e381c218f2b0d,3,33.338616,53.130341,35.018678,54.076853,24.444730,44.094011,24.585567,1970,0,14


In [15]:
y_test.head(5)

,total_fare
0,20.0
1,29.0
2,35.0
3,48.0
4,84.0


## <span style="color:#ff5f27;">⏭️ **Next:** Part 03 </span>

In the next notebook, we will train a model on the dataset we created in this notebook.